In [57]:
import numpy as np

# Gradient Descent

In [80]:
def gd(theta, grad, eta, data, max_iter=10):
    # Takes in original parameter estimate theta, gradient function grad, step-size eta
    # Returns parameter estimates obtained after max_iter steps
    thetas = np.zeros((theta.shape[0], max_iter + 1))
    thetas[:, 0] = theta.flatten()
    for i in range(max_iter):
        theta = theta + eta * grad(theta, data)
        thetas[:, i+1] = theta.flatten()
    return thetas

# Newton-Raphson

In [367]:
def nr(theta, grad, data, inverse_hessian, max_iter=10, eta = 1):
    # Takes in original parameter estimate theta, gradient function grad
    # Returns parameter estimates obtained after max_iter eta
    thetas = np.zeros((theta.shape[0], max_iter + 1))
    thetas[:, 0] = theta.flatten()
    for i in range(max_iter):
        theta = theta - eta*inverse_hessian(theta, data) @ grad(theta, data)
        thetas[:, i+1] = theta.flatten()
    return thetas

# Fisher's scoring

In [175]:
def fisher(theta, grad, data, inverse_info, max_iter=10, eta = 1):
    # Takes in original parameter estimate theta, gradient function grad
    # Returns parameter estimates obtained after max_iter eta
    thetas = np.zeros((theta.shape[0], max_iter + 1))
    thetas[:, 0] = theta.flatten()
    for i in range(max_iter):
        theta = theta - eta*inverse_info(theta, data) @ grad(theta, data)
        thetas[:, i+1] = theta.flatten()
    return thetas

# BFGS

In [176]:
def bfgs(f, theta, grad, data, max_iter=10):
    # Takes in original parameter estimate theta, negative likelihood function f, negative gradient function grad
    # Returns parameter estimates obtained after max_iter eta
    thetas = np.zeros((theta.shape[0], max_iter + 1))
    thetas[:, 0] = theta.flatten()
    I = np.diag(np.ones(theta.shape[0]))
    B = np.copy(I)
    B_inverse = np.copy(I)
    for i in range(max_iter):
        p = -B_inverse @ grad(theta, data)
        alpha = line_search(f, grad, p , theta, 1, data)
        s = alpha * p
        next_theta = (theta + s)
        y = grad(next_theta, data) - grad(theta, data)
        B = B + (y @ y.T)/(y.T @ s) - (B @ s @ s.T @ B.T)/(s.T @ B @ s)
        B_inverse = (I - (s @ y.T)/(y.T @ s)) @ B_inverse @ (I - (s @ y.T)/(y.T @ s)) + (s @ s.T)/(y.T @ s)
        theta = next_theta
        thetas[:, i+1] = theta.flatten()
    return thetas

def line_search(f, grad, p, theta, alpha, data, lower = 0, upper = float("inf"), c1 = 0.0001, c2 = 0.1):
    if f(theta, data) + c1 * alpha * p.T @ grad(theta, data) - f(theta + alpha * p, data) < 0:
        return line_search(f, grad, p, theta, 0.5*(lower + alpha), data, lower = 0, upper = alpha)
    elif -c2 * p.T @ grad(theta, data) + p.T @ grad(theta + alpha * p, data) < 0:
        return line_search(f, grad, p, theta, 2*alpha, data, lower = alpha, upper = float("inf"))
    return alpha

# Example 1 - Poisson, 1 Dimension

In [84]:
np.random.seed(0)
data = np.random.poisson(lam = 56.3, size = 1000)

In [85]:
def l1(theta, data):
    return np.sum(data)*np.log(theta) - len(data)*theta

def neg_l1(theta, data):
    return -l1(theta, data)
    
def grad1(theta, data):
    return np.sum(data)/theta - len(data)

def neg_grad1(theta, data):
    return -grad1(theta, data)

def inverse_hessian1(theta, data):
    return -(theta**2)/np.sum(data)

def inverse_info1(theta, data):
    return -(theta)/len(data)

In [86]:
theta0 = np.array([[5]])

In [87]:
%%timeit
gd(theta0, grad1, 1/len(data), data, max_iter = 100).flatten()

array([ 5.        , 15.2796    , 17.97066521, 20.10900201, 21.91361658,
       23.48726753, 24.88848342, 26.1545114 , 27.31085067, 28.37589071,
       29.36342303, 30.28411191, 31.14640855, 31.95714702, 32.72194786,
       33.44550051, 34.1317662 , 34.78412722, 35.40549894, 35.99841534,
       36.56509541, 37.10749533, 37.62734999, 38.12620654, 38.60545153,
       39.06633329, 39.50998043, 39.93741723, 40.34957665, 40.74731126,
       41.13140259, 41.50256905, 41.86147288, 42.208726  , 42.54489521,
       42.87050666, 43.18604977, 43.49198075, 43.78872555, 44.07668265,
       44.35622541, 44.6277042 , 44.89144835, 45.14776781, 45.3969547 ,
       45.63928473, 45.87501838, 46.10440209, 46.32766923, 46.54504107,
       46.75672761, 46.96292836, 47.16383303, 47.35962219, 47.55046784,
       47.73653398, 47.91797711, 48.09494666, 48.26758545, 48.43603006,
       48.6004112 , 48.76085405, 48.91747859, 49.07039983, 49.21972814,
       49.3655695 , 49.50802568, 49.64719451, 49.78317008, 49.91

In [88]:
%%timeit
gd(theta0, grad1, 10/len(data), data, max_iter = 50).flatten()

array([  5.        , 107.796     , 103.02791955,  98.50196972,
        94.2275405 ,  90.21283946,  86.46449946,  82.98717644,
        79.78316618,  76.85207592,  74.19058946,  71.79236219,
        69.6480725 ,  67.74564046,  66.07060419,  64.60662421,
        63.33606963,  62.24063228,  61.30191555,  60.50195433,
        59.82363668,  59.25101407,  58.76950095,  58.36597516,
        58.02879666,  57.74776436,  57.51402988,  57.31998507,
        57.15913627,  57.02597536,  56.91585443,  56.82486846,
        56.74974845,  56.68776607,  56.63664991,  56.5945129 ,
        56.55978994,  56.53118482,  56.50762529,  56.4882252 ,
        56.47225281,  56.45910426,  56.44828152,  56.43937398,
        56.43204329,  56.42601067,  56.42104653,  56.41696179,
        56.41360078,  56.41083535,  56.40856002])

In [89]:
%%timeit
nr(theta0, grad1, data, inverse_hessian1, max_iter=10).flatten()

array([ 5.        ,  9.55672187, 17.49404353, 29.56162586, 43.6282036 ,
       53.50662619, 56.24976704, 56.39761039, 56.398     , 56.398     ,
       56.398     ])

In [90]:
%%timeit
fisher(theta0, grad1, data, inverse_info1, max_iter=10).flatten()

array([ 5.   , 56.398, 56.398, 56.398, 56.398, 56.398, 56.398, 56.398,
       56.398, 56.398, 56.398])

In [91]:
%%timeit
bfgs(neg_l1, theta0, neg_grad1, data).flatten()

array([  5.        , 165.61875   ,  10.69020643, 144.91602358,
        44.24479993,  59.85877053,  54.42875174,  56.51883968,
        56.40221936,  56.39799096,  56.398     ])

# Example 2 - Normal Distribution, 2 Dimensions

In [141]:
np.random.seed(0)
data = np.random.normal(50, 5, size = 1000)

In [210]:
def l2(theta, data):
    return -(1/2)*np.sum(((data - theta[0])/theta[1])**2) - len(data)*np.log(theta[1])

def neg_l2(theta, data):
    return -l2(theta, data)
    
def grad2(theta, data):
    return np.array([(np.sum(data) - len(data)*theta[0])/theta[1]**2, 
                     np.sum((data - theta[0])**2/theta[1]**3) - len(data)/theta[1]])

def neg_grad2(theta, data):
    return -grad2(theta, data)

def inverse_hessian2(theta, data):
    return np.linalg.inv(np.r_[np.c_[-len(data)/theta[1]**2, 
                                     (-2*(np.sum(data) - len(data)*theta[0]))/theta[1]**3], 
                               np.c_[(-2*(np.sum(data) - len(data)*theta[0]))/theta[1]**3,
                                     -3*np.sum((data - theta[0])**2/theta[1]**4) + len(data)/theta[1]**2]])

def inverse_info2(theta, data):
    return np.linalg.inv(np.r_[np.c_[-len(data)/theta[1]**2, 
                                     0], 
                               np.c_[0,
                                     -2*len(data)/theta[1]]])

In [190]:
theta0 = np.array([10,2])[:, None]

In [177]:
#%%timeit
gd(theta0, grad2, 0.2/len(data), data, max_iter = 10000)

array([[10.        , 11.98868582, 11.99295811, ..., 49.77371493,
        49.77371494, 49.77371495],
       [ 2.        , 42.05760957, 42.05675794, ...,  4.93516579,
         4.93516579,  4.93516579]])

In [203]:
gd(np.array([44,2])[:, None], grad2, 1/len(data), data, max_iter = 100)

array([[44.        , 45.44342912, 45.50048954, 45.55744463, 45.61429336,
        45.67103454, 45.7276668 , 45.78418856, 45.84059807, 45.89689336,
        45.95307224, 46.00913231, 46.06507091, 46.12088517, 46.17657195,
        46.23212783, 46.28754915, 46.34283193, 46.39797192, 46.45296454,
        46.50780489, 46.56248774, 46.61700752, 46.6713583 , 46.72553375,
        46.7795272 , 46.83333155, 46.88693929, 46.94034251, 46.99353285,
        47.0465015 , 47.0992392 , 47.15173623, 47.2039824 , 47.25596702,
        47.30767893, 47.35910648, 47.41023751, 47.46105939, 47.51155899,
        47.56172269, 47.61153641, 47.6609856 , 47.71005526, 47.75872996,
        47.80699389, 47.85483084, 47.90222429, 47.94915739, 47.99561306,
        48.04157399, 48.08702273, 48.13194176, 48.17631349, 48.2201204 ,
        48.26334508, 48.30597032, 48.3479792 , 48.38935513, 48.43008199,
        48.4701442 , 48.50952681, 48.54821557, 48.58619704, 48.62345865,
        48.65998882, 48.69577698, 48.73081366, 48.7

In [192]:
#%%timeit
nr(theta0, grad2, data, inverse_hessian2, max_iter=10)

array([[ 1.00000000e+01, -3.23349838e+01, -1.15640238e+02,
        -2.81643301e+02, -6.13353761e+02, -1.27662782e+03,
        -2.60310263e+03, -5.25601562e+03, -1.05618233e+04,
        -2.11734294e+04, -4.23966371e+04],
       [ 2.00000000e+00,  4.06439598e+00,  8.15840678e+00,
         1.63313413e+01,  3.26699126e+01,  6.53434361e+01,
         1.30688677e+02,  2.61378257e+02,  5.22756964e+02,
         1.04551415e+03,  2.09102842e+03]])

In [204]:
nr(np.array([44,2])[:, None], grad2, data, inverse_hessian2, max_iter=10)

array([[44.        , 53.96844118, 50.57424055, 50.26822374, 50.06943558,
        49.93741953, 49.8514209 , 49.80086314, 49.77875387, 49.77396276,
        49.77371723],
       [ 2.        ,  1.27347927,  1.39499509,  1.82585984,  2.37179887,
         3.02828461,  3.74699792,  4.40152004,  4.80989846,  4.92748745,
         4.93513599]])

In [213]:
#%%timeit
fisher(theta0, grad2, data, inverse_info2, max_iter=1000)

array([[ 10.        ,  49.77371646,  49.77371646, ...,  49.77371646,
         49.77371646,  49.77371646],
       [  2.        , 202.28804783, 201.78834543, ...,   4.93516579,
          4.93516579,   4.93516579]])

In [220]:
#%%timeit
fisher(np.array([44,2])[:, None], grad2, data, inverse_info2, max_iter=30)

array([[44.        , 49.77371646, 49.77371646, 49.77371646, 49.77371646,
        49.77371646, 49.77371646, 49.77371646, 49.77371646, 49.77371646,
        49.77371646, 49.77371646, 49.77371646, 49.77371646, 49.77371646,
        49.77371646, 49.77371646, 49.77371646, 49.77371646, 49.77371646,
        49.77371646, 49.77371646, 49.77371646, 49.77371646, 49.77371646,
        49.77371646, 49.77371646, 49.77371646, 49.77371646, 49.77371646,
        49.77371646],
       [ 2.        ,  8.7114579 ,  8.37192715,  8.04567627,  7.73380208,
         7.43740659,  7.15756255,  6.89527022,  6.65140639,  6.42666857,
         6.22151895,  6.03613483,  5.87037268,  5.72375283,  5.59546926,
         5.48442526,  5.38929121,  5.30857705,  5.24070986,  5.18410739,
         5.13724021,  5.09867863,  5.06712321,  5.04142042,  5.0205662 ,
         5.00370076,  4.9900977 ,  4.97915011,  4.97035543,  4.96330059,
         4.95764809]])

In [225]:
#%%timeit
#pretty unstable
bfgs(neg_l2, theta0, neg_grad2, data)

/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


array([[ 1.00000000e+01,  3.20732160e+02,  1.10988431e+03,
         1.07689128e+03,  1.09274097e+03,  1.09274097e+03,
         1.09274097e+03,  1.09274097e+03, -1.08960272e+04,
        -7.41783081e+03,  1.08919716e+07],
       [ 2.00000000e+00,  6.26100149e+03,  9.86721009e+02,
         1.20698102e+03,  1.08211944e+03,  1.08211944e+03,
         1.08211944e+03,  1.08211944e+03, -4.15853555e+03,
        -3.47985155e+02,  5.00329071e+05]])

# Example 3 - Logistic Regression, no analytic MLE solution known

In [320]:
np.random.seed(0)
X = np.random.normal(1,1,(100,3))
theta = np.random.normal(0,5,(3,1))
probs = 1/(1+np.exp(-X @ theta))
data = np.random.binomial(1,probs)

In [365]:
def l3(theta, data):
    X = data[:, 1:]
    data = data[:, 0][:, None]
    probs = 1/(1+np.exp(-X @ theta))
    return np.sum(np.log(probs)*data + (1-data)*np.log(1-probs))

def neg_l3(theta, data):
    return -l3(theta, data)
    
def grad3(theta, data):
    X = data[:, 1:]
    data = data[:, 0][:, None]
    probs = 1/(1+np.exp(-X @ theta))
    return X.T @ (data - probs)

def neg_grad3(theta, data):
    return -grad3(theta, data)

def inverse_hessian3(theta, data):
    X = data[:, 1:]
    data = data[:, 0][:, None]
    probs = 1/(1+np.exp(-X @ theta))
    D = np.diag((probs*(1-probs)).flatten())
    return -np.linalg.inv(X.T @ D @ X)

def inverse_info3(theta, data):
    return inverse_hessian3(theta, data)

In [314]:
print(theta)

[[-6.53263426]
 [ 8.2906534 ]
 [-0.59082023]]


In [357]:
theta0 = np.array([0,0,0])[:, None]

In [371]:
gd(theta0, grad3, 1/len(data), np.c_[data, X], max_iter = 10000)

array([[ 0.00000000e+00, -2.29306593e-01, -4.30050323e-01, ...,
        -7.79602436e+00, -7.79602710e+00, -7.79602984e+00],
       [ 0.00000000e+00,  3.54170391e-01,  5.86482433e-01, ...,
         9.52118221e+00,  9.52118549e+00,  9.52118878e+00],
       [ 0.00000000e+00, -4.65952726e-03, -2.90337396e-02, ...,
        -3.01173480e-01, -3.01173494e-01, -3.01173509e-01]])

In [370]:
nr(theta0, grad3, np.c_[data, X], inverse_hessian3, max_iter=10)

array([[ 0.        , -0.97673822, -1.78581804, -2.84966575, -4.23072343,
        -5.79850151, -7.11616693, -7.71443521, -7.79902952, -7.80043537,
        -7.80043575],
       [ 0.        ,  1.17867241,  2.2146374 ,  3.54331526,  5.23097728,
         7.12567335,  8.70824779,  9.42388859,  9.52479962,  9.52647315,
         9.5264736 ],
       [ 0.        , -0.04904345, -0.15828702, -0.23880062, -0.28113835,
        -0.29834846, -0.30129451, -0.30120128, -0.30119697, -0.30119682,
        -0.30119681]])

Fisher's scoring == NR for logistic regression

In [364]:
bfgs(neg_l3, theta0, neg_grad3, np.c_[data, X], max_iter = 10)
# Unstable

/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in multiply
  """
/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in exp
  del sys.path[0]


array([[ 0.00000000e+00, -2.29306593e+01, -4.12602488e+01,
        -5.06823056e+01, -6.61759832e+01, -8.84321837e+01,
        -1.78338582e+02, -3.67771009e+01,  5.10312869e+03,
        -4.67427025e+03, -2.22920346e+03],
       [ 0.00000000e+00,  3.54170391e+01,  3.27882355e+01,
         7.76813821e+01,  8.73290332e+01,  1.16029095e+02,
         2.13774694e+02,  2.07225754e+02, -9.39679394e+03,
         9.38344879e+03, -1.59593965e+02],
       [ 0.00000000e+00, -4.65952726e-01, -1.14533920e+01,
        -6.50031642e+00, -9.23227344e+00, -6.28397364e+00,
        -1.87395565e+00,  3.47604433e+01, -1.36756546e+03,
         1.41783053e+03,  7.49850362e+01]])

# Example 3b - Logistic Regression, higher dimensions

In [372]:
np.random.seed(0)
X = np.random.normal(1,1,(10000,20))
theta = np.random.normal(0,5,(20,1))
probs = 1/(1+np.exp(-X @ theta))
data = np.random.binomial(1,probs)

In [373]:
theta0 = np.zeros(20)[:, None]

In [374]:
theta

array([[ 0.19754948],
       [ 1.69188807],
       [-4.21091597],
       [-0.24816141],
       [-6.1512268 ],
       [-4.94396336],
       [ 3.01037293],
       [-0.79774301],
       [-6.41673982],
       [-6.1911004 ],
       [ 3.58614352],
       [-2.44454335],
       [ 8.36310749],
       [ 2.71818862],
       [ 2.20786495],
       [-1.95951649],
       [-6.57447333],
       [ 2.7656804 ],
       [ 1.00997219],
       [10.16281831]])

In [375]:
gd(theta0, grad3, 1/len(data), np.c_[data, X], max_iter = 10000)

array([[ 0.        , -0.07417171,  0.20031893, ...,  0.20911615,
         0.20911966,  0.20912317],
       [ 0.        , -0.0460053 ,  0.25562308, ...,  1.6569537 ,
         1.65698025,  1.65700679],
       [ 0.        , -0.16422751,  0.03472365, ..., -4.13593676,
        -4.13600258, -4.13606839],
       ...,
       [ 0.        , -0.02736347,  0.2918735 , ...,  2.7113574 ,
         2.7113997 ,  2.711442  ],
       [ 0.        , -0.06133535,  0.22724388, ...,  0.9613838 ,
         0.9613992 ,  0.96141459],
       [ 0.        ,  0.10469057,  0.54108137, ...,  9.84123434,
         9.84138986,  9.84154537]])

In [376]:
# Inverting a 20 x 20 matrix takes some time
nr(theta0, grad3, np.c_[data, X], inverse_hessian3, max_iter=10)

array([[ 0.00000000e+00,  1.44713215e-02,  2.84900769e-02,
         4.67882851e-02,  7.13480329e-02,  1.05741073e-01,
         1.51437871e-01,  1.99039049e-01,  2.28842617e-01,
         2.36155857e-01,  2.36491042e-01],
       [ 0.00000000e+00,  1.18575794e-01,  2.25207680e-01,
         3.72254571e-01,  5.81915577e-01,  8.71577163e-01,
         1.23176805e+00,  1.58741516e+00,  1.80699366e+00,
         1.86156173e+00,  1.86411452e+00],
       [ 0.00000000e+00, -3.15274690e-01, -5.98408820e-01,
        -9.73906183e-01, -1.49388767e+00, -2.20167272e+00,
        -3.08275332e+00, -3.96128579e+00, -4.50751716e+00,
        -4.64355288e+00, -4.64991485e+00],
       [ 0.00000000e+00, -4.23247685e-03, -1.08223489e-02,
        -2.20157914e-02, -4.19353173e-02, -7.56194813e-02,
        -1.24724340e-01, -1.75554233e-01, -2.05927595e-01,
        -2.13175031e-01, -2.13505785e-01],
       [ 0.00000000e+00, -4.48268002e-01, -8.64889585e-01,
        -1.41575586e+00, -2.16443311e+00, -3.16625141e+00,
  

In [377]:
bfgs(neg_l3, theta0, neg_grad3, np.c_[data, X], max_iter = 10)

/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.
/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in multiply
  """
/Users/erictay1997/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in exp
  del sys.path[0]


array([[ 0.00000000e+00, -7.41717103e+02,  5.00764549e+02,
         5.17750520e+02, -1.59358685e+02, -4.29627215e+02,
        -1.27263126e+05,  7.89890218e+04,  6.51271154e+04,
         1.30674407e+04, -1.10204587e+05],
       [ 0.00000000e+00, -4.60052998e+02,  2.24874084e+03,
         2.11469268e+03,  8.39608308e+01, -4.06890685e+03,
        -2.12342265e+05,  9.10551716e+04,  1.91600980e+05,
         9.46649523e+04, -4.61340826e+05],
       [ 0.00000000e+00, -1.64227508e+03, -4.89902138e+03,
        -4.43222453e+03, -1.04583828e+03,  1.04129486e+04,
         1.25661440e+05,  4.46031922e+04, -3.14555109e+05,
        -2.32581080e+05,  9.45452057e+05],
       [ 0.00000000e+00, -8.90882239e+02, -4.55539089e+02,
        -3.16346982e+02, -2.20631092e+01,  2.54320527e+03,
        -6.96012349e+04,  7.30578767e+04, -2.33607060e+04,
        -4.45696460e+04,  1.39437311e+05],
       [ 0.00000000e+00, -1.91044054e+03, -6.50419091e+03,
        -5.96572344e+03, -1.71885914e+03,  1.22533265e+04,
  